In [1]:
import pandas as pd
from googleapiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors
from datetime import datetime
import requests

In [2]:
# Set up the API key and YouTube API
API_KEY = open('youtube_api_3.txt', 'r').read()
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [3]:
def check_comments_enabled(video_id):
    video_url = f'https://www.googleapis.com/youtube/v3/videos?part=status&id={video_id}&key={API_KEY}'
    video_response = requests.get(video_url)
    video_data = video_response.json()
    print(video_data)
    if 'items' in video_data and len(video_data['items']) > 0:
        return not video_data['items'][0]['status'].get('commentsDisabled', False)
    else:
        print(f'Video not found for ID: {video_id}')
        return False  # Default to False if video not found

In [4]:
def search_videos(query, max_results=50):
    """Search for videos on a specific topic."""
    search_response = youtube.search().list(
        q=query,
        type='video',
        part='id',
        maxResults=max_results
    ).execute()
    video_ids = [item['id']['videoId'] for item in search_response['items']]
    return video_ids

In [5]:
def get_comments(video_id):
    """Get comments for a given video ID."""
    comments = []

    try: # Wrap the API call in a try-except block
        # Retrieve the comments
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText',
            maxResults =50,
            order = 'relevance' #,
            #searchTerms = 'Team Fortress 2'
        ).execute()

        if response:
            for item in response['items']:
                text = item['snippet']['topLevelComment']['snippet']['textDisplay']
                channel_id = item['snippet']['topLevelComment']['snippet']['channelId']
                like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
                channel_id_text = item['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
                date = item['snippet']['topLevelComment']['snippet'].get('publishedAt', 'N/A')

                if date != 'N/A':
                  date = datetime.fromisoformat(date[:-1])
                else:
                  date = None

                comment = {
                    'video_id': video_id,
                    'text': text,
                    'channel_id': channel_id,
                    'like_count': like_count,
                    'channel_id_text': channel_id_text,
                    'date': date
                }
                comments.append(comment)

    except googleapiclient.errors.HttpError as e:
        # Handle the HttpError, specifically when comments are disabled
        if e.resp.status == 403 and "commentsDisabled" in str(e):
            print(f"Comments are disabled for video ID: {video_id}")
        elif e.resp.status == 404: # Check for video not found error
            print(f"Video not found for video ID: {video_id}")
        else:
            # Re-raise the exception for other errors
            raise e
    return comments

In [6]:
def get_video_info(video_id):
  """Get video information for a given video ID."""
  # Step 1: Get video details
  video_url = f'https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id={video_id}&key={API_KEY}'
  video_response = requests.get(video_url)
  video_data = video_response.json()

  if 'items' in video_data and len(video_data['items']) > 0:
      channel_id = video_data['items'][0]['snippet']['channelId']
      channel_name = video_data['items'][0]['snippet']['channelTitle']
      views = video_data['items'][0]['statistics'].get('viewCount', 'N/A')
      likes = video_data['items'][0]['statistics'].get('likeCount', 'N/A')
      published_date = video_data['items'][0]['snippet'].get('publishedAt', 'N/A')

      # Step 2: Get channel details
      channel_url = f'https://www.googleapis.com/youtube/v3/channels?part=statistics&id={channel_id}&key={API_KEY}'
      channel_response = requests.get(channel_url)
      channel_data = channel_response.json()

      if 'items' in channel_data and len(channel_data['items']) > 0:
          subscribers = channel_data['items'][0]['statistics'].get('subscriberCount', 'N/A')
      else:
          subscribers = 'N/A'
  else:
      print('Video not found or no data available.')
      channel_name, views, likes, subscribers, published_date = 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'


  # Step 3: Create a list
  data = {
      'Video ID': video_id,
      'Channel Name': channel_name,
      'Views': int(views) if views != 'N/A' else -1,
      'Likes': int(likes)if likes != 'N/A' else -1,
      'Subscribers': int(subscribers) if subscribers != 'N/A' else -1,
      'published_date': datetime.fromisoformat(published_date[:-1]) if published_date != 'N/A' else None
  }
  return data

In [7]:
def perform_data_collection(topic,max_results):
  # Get the top  video IDs
  print(f"Searching for videos about: {topic}")
  video_ids = search_videos(topic, max_results=max_results)

  all_comments = []
  video_info_list = []

  # Iterate through the video IDs to get comments
  for video_id in video_ids:
    # Get video information
    video_info_list.append(get_video_info(video_id))
    print(f"Fetching comments for video ID: {video_id}")
    comments = get_comments(video_id)
    all_comments.extend(comments)  # Add the comments to the list
    print("length of all comments ", len(all_comments))
  return all_comments, video_info_list

In [8]:
video_list_full = []
comment_list_full = []

In [9]:
df = pd.read_csv('GamesList.csv')

# Store the titles in a variable called 'topic'
topics = df['title'].tolist()  # Convert the 'title' column to a list

# Iterate through the titles
for topic in topics:
    print(topic)
    max_results = 200
    all_comments, video_info_list = perform_data_collection(topic,max_results)
    video_list_full.extend(video_info_list)
    comment_list_full.extend(all_comments)

Snakeybus
Searching for videos about: Snakeybus
Fetching comments for video ID: Vb1gZyckHX8
length of all comments  50
Fetching comments for video ID: xzDyhUdsuCc
length of all comments  100
Fetching comments for video ID: UNhGynJEp-M
length of all comments  150
Fetching comments for video ID: xwj8mD5MnMg
length of all comments  200
Fetching comments for video ID: zZSbH9bUVaM
length of all comments  250
Fetching comments for video ID: AsV7_nJHd-A


length of all comments  300
Fetching comments for video ID: BsjS8bmMFNU
Comments are disabled for video ID: BsjS8bmMFNU
length of all comments  300
Fetching comments for video ID: FJ5AG1zv0pY
length of all comments  311
Fetching comments for video ID: ZZx3S4GlcHQ
length of all comments  311
Fetching comments for video ID: A3_Xhge3qlE
length of all comments  361
Fetching comments for video ID: ATfxLkKqBYQ
length of all comments  411
Fetching comments for video ID: RaWn4YXZpP8
length of all comments  461
Fetching comments for video ID: 3a2Ru1d5YCc
length of all comments  511
Fetching comments for video ID: E4vzdt4wTXc
length of all comments  561
Fetching comments for video ID: dedWvlqgeq4
length of all comments  611
Fetching comments for video ID: Whyndp_iyZM
length of all comments  622
Fetching comments for video ID: 83tvhlrT2lc
length of all comments  672
Fetching comments for video ID: AuEAoD8pYRk
length of all comments  722
Fetching comments for video ID: NtzkdSPMfOc
length of all co

Fetching comments for video ID: smlef-jkmuQ
Comments are disabled for video ID: smlef-jkmuQ
length of all comments  0
Fetching comments for video ID: uXhfnGv_fEQ
length of all comments  5
Fetching comments for video ID: xUko31SmXxA
length of all comments  55
Fetching comments for video ID: Ta9ReKaZOGw
length of all comments  105
Fetching comments for video ID: YSlrAuFAnhc
length of all comments  116
Fetching comments for video ID: hV_nResY_nE


length of all comments  166
Fetching comments for video ID: wgR1tNgLV68
Comments are disabled for video ID: wgR1tNgLV68
length of all comments  166


Fetching comments for video ID: Aj7tJ79EiQU
Comments are disabled for video ID: Aj7tJ79EiQU
length of all comments  166
Fetching comments for video ID: cs3BOop15nU
length of all comments  167
Fetching comments for video ID: wrE8wHZQ1BI
length of all comments  181
Fetching comments for video ID: TjApDcGW9Io
length of all comments  231
Fetching comments for video ID: UpNid_rWDnI
length of all comments  281
Fetching comments for video ID: 9Lypkwll6tM
length of all comments  326
Fetching comments for video ID: vSbDWlCPloM
length of all comments  376
Fetching comments for video ID: HSnfcBp31qc
length of all comments  426
Fetching comments for video ID: G9SdoVNn4fs
length of all comments  476
Fetching comments for video ID: GTfYP-Y7bpM
length of all comments  526
Fetching comments for video ID: coXBnLnaMYo
length of all comments  576
Fetching comments for video ID: 82v8it8OAK4
length of all comments  621
Fetching comments for video ID: _H3-AXp8EhI
length of all comments  624
Fetching comment

...

Comments are disabled for video ID: 0JG5Y7ZWvWU
length of all comments  98
Fetching comments for video ID: rmTcHE-JPdg
length of all comments  148
Fetching comments for video ID: _PBhl3sxgfE
length of all comments  198
Fetching comments for video ID: PafEXxg3TK0
length of all comments  211
Fetching comments for video ID: EUPpdfBuGfM
length of all comments  261
Fetching comments for video ID: N3lrwiwFWQQ
length of all comments  261
Fetching comments for video ID: ILR2wEY_e_Q
length of all comments  311
Fetching comments for video ID: 6nK4PjseyQE
length of all comments  361
Fetching comments for video ID: 3RXb_oSU-4A
length of all comments  411
Fetching comments for video ID: Tg8TxsFf-H8
length of all comments  461
Fetching comments for video ID: EdVzL6xpwvo
length of all comments  511
Fetching comments for video ID: ScejdNGTXCU
length of all comments  561
Fetching comments for video ID: 4THY_VrsraE
length of all comments  570
Fetching comments for video ID: cKqNLkAyUb4
length of all com

Fetching comments for video ID: nYtQxseF5tE
Comments are disabled for video ID: nYtQxseF5tE
length of all comments  639
Fetching comments for video ID: InTwZnuKB0w
length of all comments  689
Fetching comments for video ID: ZXD_5qWVxW4
length of all comments  739
Fetching comments for video ID: 7nv4x9pVIAY
length of all comments  789
Fetching comments for video ID: fABPqSAJptc
length of all comments  839
Fetching comments for video ID: rpNTSBQj84Q
length of all comments  889
Fetching comments for video ID: DfWbZACatF0


length of all comments  939
Fetching comments for video ID: QhhTdi-xmn0
Comments are disabled for video ID: QhhTdi-xmn0
length of all comments  939
Fetching comments for video ID: dKwq0zJbdGA
length of all comments  971
Fetching comments for video ID: kuYle1Fgw-Y
length of all comments  971
Fetching comments for video ID: mMNnvuEzBt8
length of all comments  1021
Fetching comments for video ID: fKGBwP5wiEM
length of all comments  1045
Fetching comments for video ID: bDP5xYtNNAU
length of all comments  1088
Fetching comments for video ID: AJLgkqTKsEE
length of all comments  1105
Fetching comments for video ID: von0Ux2c2ac
length of all comments  1155
Fetching comments for video ID: 4wPhuiqqL08
length of all comments  1205
Fetching comments for video ID: 92vkmjd_oYU
length of all comments  1255
Fetching comments for video ID: 4xw45qIkcv8
length of all comments  1305
Fetching comments for video ID: rSXGccP_kwM
length of all comments  1355
Fetching comments for video ID: vEyUu-u2aFA
length 

length of all comments  2101
Fetching comments for video ID: mq8j61itmuw
Comments are disabled for video ID: mq8j61itmuw
length of all comments  2101
Fetching comments for video ID: PJVFvWod3tI
length of all comments  2151
Fetching comments for video ID: VmO7kLxy4u0
length of all comments  2166
ibb & obb
Searching for videos about: ibb & obb
Fetching comments for video ID: 2iif636QAcc
length of all comments  50
Fetching comments for video ID: di9bRFTR3Lc
length of all comments  100
Fetching comments for video ID: ayk_00tFqMQ
length of all comments  122
Fetching comments for video ID: eoeacno-5aw
length of all comments  135
Fetching comments for video ID: quGCVezY1hY
length of all comments  154
Fetching comments for video ID: KUdxShTNWTc


length of all comments  161
Fetching comments for video ID: hlPoZsjLFyc
Comments are disabled for video ID: hlPoZsjLFyc
length of all comments  161
Fetching comments for video ID: Vc0TfUn899o
length of all comments  175
Fetching comments for video ID: ws-dkTUlup4
length of all comments  176
Fetching comments for video ID: WbvdCIFCueY
length of all comments  223
Fetching comments for video ID: o5VH06YtUZQ
length of all comments  273
Fetching comments for video ID: vMTIvxJ7bFs
length of all comments  280
Fetching comments for video ID: -uhEKIQsJRE
length of all comments  285
Fetching comments for video ID: LAO6JZW0a94
length of all comments  287
Fetching comments for video ID: vCmuHjPcc1M
length of all comments  337
Fetching comments for video ID: dUqxrF96HXw
length of all comments  343
Fetching comments for video ID: dMvzLrPq1oo
length of all comments  344
Fetching comments for video ID: 6iPvgPvADcA
length of all comments  375
Fetching comments for video ID: yy54TL_J8Qc
length of all co

In [11]:
# Convert all_comments to a df with columns as keys in the dictionary and values as each row
comments_df = pd.DataFrame(comment_list_full)
# Create a dataframe for the video information
video_info_df = pd.DataFrame(video_list_full)

In [19]:
# Ignore duplicate videos(not done for comments as the api fetches only unique comments)
video_info_df.drop_duplicates(subset='Video ID', inplace=True)

In [22]:
comments_df.to_csv('youtube_comments.csv', index=False)
comments_df.shape

(69374, 6)

In [23]:
result_df = video_info_df.groupby('Channel Name', as_index=False)[['Views', 'Likes']].sum().sort_values(by=['Views', 'Likes'], ascending=False)
video_info_df.to_csv('youtube_videos.csv', index=False)